# Multi-Agent Supervisor
---

### What is Multi-Agent Supervisor?

A Multi-Agent Supervisor is a control mechanism that oversees and coordinates multiple autonomous agents operating within a system. It ensures that agents work collaboratively and efficiently by managing tasks, resolving conflicts, optimizing resource allocation, and enforcing system constraints. The supervisor can be centralized, decentralized, or distributed, depending on the system architecture. It is commonly used in multi-robot systems, industrial automation, and AI-driven applications to enhance coordination, adaptability, and decision-making.

As the number of agents increases, the branching logic also becomes more complex. The Supervisor agent gathers various specialized agents together and operates them as a single team. The Supervisor agent observes the progress of the team and performs logic such as calling the appropriate agent for each step or terminating the task.

**Reference**

- [Multi-Agent Supervisor Concept](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor)  
- [LangChain `create_react_agent` built-in function](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent)

In [ ]:
import os
from dotenv import load_dotenv
from azure_genai_utils.tracer import get_langchain_api_key, set_langsmith

load_dotenv(override=True)

# If you want to trace your RAG API calls, please set the tracing=True. You need to have a valid Langchain API key.
langchain_key, has_langchain_key = get_langchain_api_key()
set_langsmith("[RAG Innv Lab] 1_Agentic-Design-Pattern", tracing=False)

azure_openai_chat_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

<br>

## 🧪 Step 1. Test and Construct each module
---

### Define your LLM
This hands-on only uses the `gpt-4o-mini`, but you can utilize multiple models in the pipeline.

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=azure_openai_chat_deployment_name, temperature=0)

### Tools

Before building the entire the graph pipeline, we will test and construct each module separately.

- **Researcher**
- **Coder**

In [ ]:
from azure_genai_utils.tools import BingSearch
from langchain_experimental.tools import PythonREPLTool

WEB_SEARCH_FORMAT_OUTPUT = True

web_search_tool = BingSearch(
    max_results=3,
    locale="en-US",
    include_news=False,
    include_entity=False,
    format_output=WEB_SEARCH_FORMAT_OUTPUT,
)

python_repl_tool = PythonREPLTool()

In [ ]:
web_search_tool.invoke("Where is Seoul?")

In [ ]:
python_repl_tool.invoke("print('Hello, World!')")

### ReAct agent test (Not required. Just for testing)

In [ ]:
from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(llm, tools=[web_search_tool])
research_agent.invoke({"messages": "Where is Seoul?"})

In [ ]:
coder_agent = create_react_agent(llm, tools=[python_repl_tool], prompt=None)
coder_agent.invoke({"messages": [("user", "print 'Hello, World!'")]})

<br>

## 🧪 Step 2. Define the Graph
---

### State Definition

- `messages`: Messages to be passed between agents
- `next`: Next agent to be called

In [ ]:
import operator
from typing import Sequence, Annotated
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: Annotated[str, "Next agent to be called"]

### Create Agent Supervisor

Create a supervisor agent that manages the agents.

In [ ]:
from pydantic import BaseModel
from typing import Literal

# Member agents
members = ["Researcher", "Coder"]

# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options_for_next = ["FINISH"] + members


class RouteResponse(BaseModel):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options_for_next]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def supervisor_agent(state: AgentState):

    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        " following workers: {members}. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next? "
                "Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options_for_next), members=", ".join(members))

    supervisor_chain = prompt | llm.with_structured_output(RouteResponse)
    return supervisor_chain.invoke(dict(state))

Not required, but it is good to test the agent before creating the graph workflow.

In [ ]:
supervisor_agent({"messages": [], "next": "Researcher"})

### Create Agents

Create agents that perform sub-tasks.
- `Researcher`: Researches the topic
- `Coder`: Codes the solution

In [ ]:
import functools
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_core.prompts import load_prompt


# Agent node that invokes the agent with the given state
def agent_node(state: AgentState, agent, name) -> AgentState:
    agent_response = agent.invoke(state)
    # Return the last message of the agent as a HumanMessage and set the next agent
    return {
        "messages": [
            HumanMessage(content=agent_response["messages"][-1].content, name=name)
        ],
        "next": "Supervisor",  # Set the next agent to Supervisor or any other logic
    }


# Create Research Agent
research_agent = create_react_agent(llm, tools=[web_search_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Create Coder Agent
code_system_prompt = load_prompt("../../../prompts/code-system-prompt-kr.yaml").format()
coder_agent = create_react_agent(
    llm, tools=[python_repl_tool], prompt=code_system_prompt
)
coder_node = functools.partial(agent_node, agent=coder_agent, name="Coder")

### Construct the Graph

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver


def get_next(state):
    return state["next"]


workflow = StateGraph(AgentState)

# Node definition
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", coder_node)
workflow.add_node("Supervisor", supervisor_agent)

# Edge definition
workflow.add_edge(START, "Supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("Supervisor", get_next, conditional_map)

for member in members:
    workflow.add_edge(member, "Supervisor")

# Compile the workflow
app = workflow.compile(checkpointer=MemorySaver())

### Visualize the graph

In [ ]:
from azure_genai_utils.graphs import visualize_langgraph

visualize_langgraph(app, xray=True)

<br>

## 🧪 Step 3. Execute the Graph
---

### Execute the graph

In [ ]:
from langchain_core.runnables import RunnableConfig
from azure_genai_utils.messages import stream_graph, invoke_graph, random_uuid

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
inputs = {
    "messages": [
        HumanMessage(
            content="Visualize Microsoft's stock price over the past 5 years on a graph."
        )
    ],
}

invoke_graph(app, inputs, config)